In [988]:
# Importing Libraries

import json
import re

# Opening Database.json file to get Inventory data

file = open("DataBase.json", 'r')
items = file.read()
file.close()
Items = json.loads(items)

# Initialising Dictionaries and Variables

Cart = {}
cartCounter = 1

In [989]:
# Creating a function for dumping/updating Inventory Database

def dumpItems():
  items = json.dumps(dict(sorted(Items.items())))
  file = open("DataBase.json", 'w')
  file.write(items)
  file.close()

In [990]:
# Creating a function to display items in inventory in Tabular Format

def viewInventory():
  print("\nThe Items in the Inventory are :- ")
  print("{:<12} {:<35} {:<12} {:<12}".format("Item ID","Item Name","Price","Quantity"))
  print("{:<12} {:<35} {:<12} {:<12}".format("-------","---- ----","-----","--------"))
  [print("{:<12} {:<35} {:<12} {:<12}".format(key,value["Name"],value["Price"],value["Quantity"])) for key, value in sorted(Items.items())]

In [991]:
# I have also added Cart feature same as Online Shopping websites
# Creating a function to display items in the Cart in Tabular Format

def viewCart():
  print("\nThe Items in your Cart are :- \n")
  print("{:<12} {:<12} {:<35} {:<12} {:<12}".format("Item No.","Item ID","Item Name","Rate","Quantity"))
  print("{:<12} {:<12} {:<35} {:<12} {:<12}".format("--------","-------","---------","-----","--------"))
  [print("{:<12} {:<12} {:<35} {:<12} {:<12}".format(key,value["Item ID"],value["Name"],value["Price"],value["Quantity"])) for key, value in sorted(Cart.items())]

In [992]:
# This function will construct bills

def bill():
  global Bill
  file = open("Sales.json", 'r')
  sales = file.read()
  file.close()

# I have added Transaction ID Feature

  regx = re.findall('Transaction ID = +\d+',sales)
  print("Your Invoice is :-")
  if len(regx) > 0:
    TS = "----------------------------- Transaction ID = %s ------------------------------\n" % (int(regx[-1][-1:-6:-1][::-1])+1)
  else:
    TS = "----------------------------- Transaction ID = 10001 ------------------------------\n"
  
  Bill = [
  TS,
  "{:<12} {:<12} {:<35} {:<12} {:<12}".format("--------","-------","---------","-----","--------"),
  "{:<12} {:<12} {:<35} {:<12} {:<12}".format("Item No.","Item ID","Item Name","Quantity","Rate"),
  "{:<12} {:<12} {:<35} {:<12} {:<12}".format("--------","-------","---------","-----","--------"),
  ["{:<12} {:<12} {:<35} {:<12} {:<12}".format(key,value["Item ID"],value["Name"],value["Quantity"],value["Price"]) for key, value in Cart.items()],
  "{:<12} {:<12} {:<35} {:<12} {:<12}".format("--------","-------","---------","-----","--------"),
  ]

# Costumer Invoice will be displayed here

  for i in range(len(Bill)):
    if i == 4:
      for j in range(len(Bill[i])):
        print(Bill[i][j])
    else:
      print(Bill[i])
  total = sum(int(value["Price"]) for key,value in Cart.items())
  print("\nSub Total = ",total)

# Transactions will be appended in Sale.json over here

  file = open("Sales.json", 'a')
  file.write("\n")
  for i in range(len(Bill)):
    if i == 4:
      for j in range(len(Bill[i])):
        file.write((Bill[i][j])+"\n")
    else:
      file.write((Bill[i])+"\n")
  totalAmount = "TOTAL AMOUNT RECEIVED ===> "+str(total)

# End of Transction

  file.write(totalAmount)
  file.close()
  print("\n\nTRANSACTION SUCCESSFUL\nVISIT OUR INVENTORY BACK SOON :)")

In [993]:
# This funciton handles the purchase operations

def Purchase():
  global cartCounter
  itemId = input("Enter the Item ID :- ")
  if itemId in Items:
    print("Name of item -\t\t{}\nPrice -\t\t\t{}\nStock Available -\t{}".format(Items[itemId]['Name'],Items[itemId]['Price'],Items[itemId]['Quantity']))
    purchaseQuantity = int(input("\nEnter the amount of %s's you want to purchase :- " % Items[itemId]['Name']))
    if purchaseQuantity <= int(Items[itemId]['Quantity']):
      Cart[cartCounter] = {"Item ID":itemId,"Name":Items[itemId]['Name'],"Price":str(int(Items[itemId]['Price'])*purchaseQuantity),"Quantity":purchaseQuantity}
      cartCounter = cartCounter + 1
      Items[itemId]['Quantity'] = str(int(Items[itemId]['Quantity']) - purchaseQuantity)
      print("Items Are added in your cart.")    
    else:
      print("Please try later.\nWe only have %s %s's in our Inventory." % (Items[itemId]['Quantity'],Items[itemId]['Name']))
  else:
    print("Item not in the Inventory...")

In [994]:
# This function is used to add Items to the Inventory

def add():
  print("You are about to add items to the Inventory\n")
  itemId = input("Enter the Item ID :- ")
  itemName = input("Enter the Item Name :- ")
  itemPrice = input("Enter the Item Price :- ")
  itemQuantity = input("Enter the Item Quantity :- ")
  if (len(itemId)>0 and len(itemName)>0 and len(itemPrice)>0 and len(itemQuantity)>0):
    Items[itemId] = {"Name":itemName,"Price":itemPrice,"Quantity":itemQuantity}
  else:
    print("You have missed entering a value, Retry.")

In [995]:
# This function handles Checkout and Billing Process

def checkout():
  print("The items in your Cart are :- ")
  viewCart()
  confirm = input("CONFIRM CHECKOUT\nAre you sure you want to checkout? If yes, enter \"YES\", else press any KEY.\n---> ")
  if confirm == "YES":
    total = sum(int(value["Price"]) for key,value in Cart.items())
    print("Your Total is :- ",total)
  else:
    print("Back to Menu...")
    pass
  while True:
    paid = input("Have you paid? If yes, enter \"YES\" to receive the Invoice.\n--->")
    if paid == "YES":
      bill()
      break

In [996]:
# The Main Function/Driver Code of the whole IMS

viewInventory()

while True:
  operation = input("\nTo Purchase Items\t - Enter 1\nAdd Items to Inventory\t - Enter 2\nView Inventory\t\t - Enter 3\nView Cart\t\t - Enter 4\nCheckout\t\t - Enter 5\nExit\t\t\t - Enter 0\n\nWhat would you like to do? :- ")
  if operation == "1":
    Purchase()
    dumpItems()
  elif operation == "2":
    add()
    dumpItems()
  elif operation == "3":
    viewInventory()
  elif operation == "4":
    viewCart()
  elif operation == "5":
    checkout()
    break
  elif operation == "0":
    print("Program Terminated")
    break
  else:
    print("\nSelect Valid Operation")


The Items in the Inventory are :- 
Item ID      Item Name                           Price        Quantity    
-------      ---- ----                           -----        --------    
1            Cadbury Perk Home Treats            66           27          
10           Amul Dark Chocolate                 94           27          
11           Amul 99% Cacao Chocolate            298          40          
12           Cadbury Dairy Milk Silk Bar         70           39          
13           Cadbury Bournville Raisin & Nuts    324          35          
14           Amul Sugar Free Dark Chocolate      124          12          
15           Lotte Choco Pie                     180          38          
16           Cadbury Gems Minis Home Treats      100          20          
17           Cadbury Choclairs Gold Pack         50           23          
18           Amul Dark Assorted Pack             347          47          
19           Cadbury Rakhi Special Potli         629          21